In [ ]:
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.utils import upload_to_neuroglancer as upload
import numpy as np
import tifffile as tf
from pathlib import Path
import os

## Converting image data to neuroglancer precomputed format

Image data will be assumed to be stored locally in octree format and at multiple resolutions, such as
```default.0.tif 0/default.0.tif 1/default.0.tif ... 8/default.0.tif```.

A user only needs to specity a path to the octree top level, and specify the number of resolutions to use.

### The octree path can be modified to generated files from different data

In [ ]:
directory = Path("..") / ".." / ".." / "tests" # working directory containing octree data, modify this
top_level = directory / "data_octree" # top level of octree data, modify thiss
num_res = 2

After defining file paths, we call `get_volume_info` to set up the image channel parameters appropriately.

In [ ]:
(ordered_files, 
 bin_paths, 
 vox_size, 
 tiff_dims) = upload.get_volume_info(os.fspath(top_level), num_res, 0)

### The URL-formated layer location parameter can be modified to send generated files to
 - any location on a local machine by using the "file://" prefix and then filepath
 - a google storage account by using the "gs://" prefix and then url
 - an aws account by using the "s3://" prefix and then url

In [ ]:
layer_location = "file://" + os.fspath(directory) + "/test_precomputed/" # file upload destination, modify this
vols = upload.create_image_layer(
    layer_location,
    tiff_dims, vox_size, num_res
)

In [ ]:
%%capture 
# to suppress output
for i in range(num_res):
    upload.upload_chunks(
        vols[i], ordered_files[i], bin_paths[i], parallel=False
    )

The end result is a set of folders, 1 per resolution, wherever we specified when defining our image layer, named by resolution.  

## Converting swc data to neuroglancer precomputed

SWC data will be assumed to be stored locally in `.swc` format, such as
```default.0.swc```.

As before, this tutorial simply shows how to generate the requisite files, and the user can provide any destination for said files. The dafault is a local folder, but this can be modified to any url.


In [ ]:
from brainlit.utils import upload_skeleton

### The octree path can be modified to generated files from different data

We use `get_volume_info` from the `upload_skeleton` module instead, defining a volume in the same way.

In [ ]:
swcpath = directory / "data_swcs" # swc data location, modify this
origin, vox_size, tiff_dims = upload_skeleton.get_volume_info(os.fspath(top_level), num_res)

### The URL-formated layer location parameter can be modified to send generated files to different locations

In [ ]:
layer_location_swc = "file://" + os.fspath(directory) + "/test_precomputed/" # file upload destination, modify this
vol = upload_skeleton.create_skeleton_layer(
    layer_location_swc, 
    vox_size, tiff_dims, num_res
)

First, the swc data is converted to the correct data type (Skeleton object). Then it is uploaded using similar methods as the image data.

In [ ]:
skels, segids = upload_skeleton.create_skel_segids(
    os.fspath(swcpath), origin
)
for skel in skels:
    vol.skeleton.upload(skel)

This will create a `skeletons` folder containing the swcs in precomputed format. Again, any destination url can be used other than `file://path`.